In [66]:
import numpy as np

In [ ]:
def precision(relevance_arr: np.array) -> float:
    return np.mean(relevance_arr)


def precision_at_k(relevance_arr: np.array, k: int) -> float:
    return np.mean(relevance_arr[0:k])


def recall_at_k(relevance_arr: np.array, number_relevant_doc: int, k: int) -> float:
    return relevance_arr[0:k].sum() / number_relevant_doc


def average_precision(relevance_arr: np.array):
    """Calcula la presicion promedio, recuerda que solo se calcula donde
    hay documentos relevantes

    Args:
        relevance_arr (np.array): _description_

    Returns:
        _type_: _description_
    """

    precisiones = [
        precision_at_k(relevance_arr, k + 1)
        for k in range(len(relevance_arr))
        if relevance_arr[k] == 1
    ]
    if len(precisiones) == 0:
        return np.float64(0)
    else:
        return np.mean(precisiones)


def mean_average_precision(querys_matrix: np.ndarray) -> float:

    # scores = [average_precision(querys_matrix) for row in querys_matrix]
    querys_matrix = np.array(querys_matrix, dtype=object)
    arr_result = np.zeros(len(querys_matrix))
    for i, row in enumerate(querys_matrix):
        arr_result[i] = average_precision(row)  # cada row puede tener tamaño distinto
    return np.mean(arr_result)
def dct_at_one_k(relevance_arr: np.array, k:int) -> float:

    gain = np.sum(relevance_arr)
    discount_factor = np.log2(max(k,2))
    return gain/discount_factor

def dcg_at_k(relevance_arr: np.array, k:int) -> float:
    sum = 0
    for i in range(1,k+1):
        sum+=dct_at_one_k(relevance_arr[0:i],k)
    print(sum)
    return sum
def ndcg_at_k(relevance_arr: np.array, k:int) -> float:
    dcg = dcg_at_k(relevance_arr, k)
    ideal_dcg = dcg_at_k(sorted(relevance_arr, reverse=True), k)
    return dcg / ideal_dcg


In [ ]:
def test_dcg_at_k():
    rels1 = np.array([1, 1, 1, 1])
    expected1 = 1/1 + 1/1.585 + 1/2 + 1/2.322
    assert np.isclose(dcg_at_k(rels1, 4), expected1, atol=1e-6)

    rels2 = np.array([1, 0, 0, 0])
    assert np.isclose(dcg_at_k(rels2, 4), 1.0, atol=1e-6)

    rels3 = np.array([3, 2, 3, 0, 1])
    expected3 = 3/1 + 2/1.585 + 3/2 + 0 + 1/2.585
    assert np.isclose(dcg_at_k(rels3, 5), expected3, atol=1e-6)

In [89]:
rels1 = np.array([1, 1, 1, 1])

In [95]:
expected1 = 1/1 + 1/1.585 + 1/2 + 1/2.322

In [96]:
expected1

2.5615780478593186

In [87]:
test_dcg_at_k()

TypeError: unsupported operand type(s) for -: 'NoneType' and 'float'

In [ ]:
# Pruebas


def test_precision():
    assert precision(np.array([0, 0, 0, 1])) == np.float64(0.25)
    assert precision(np.array([1, 1, 1, 1])) == np.float64(1.0)
    assert precision(np.array([0, 0, 0, 0])) == np.float64(0.0)
    assert precision(np.array([1, 0, 1, 0])) == np.float64(0.5)


def test_precision_at_k():
    assert precision_at_k(np.array([0, 0, 0, 1]), 2) == np.float64(0.0)
    assert precision_at_k(np.array([1, 1, 1, 1]), 2) == np.float64(1.0)
    assert precision_at_k(np.array([1, 0, 1, 0]), 2) == np.float64(0.5)
    assert precision_at_k(np.array([1, 0, 1, 0]), 3) == np.float64(2 / 3)


def test_recall_at_k():
    arr = np.array([0, 0, 0, 1])
    assert recall_at_k(arr, number_relevant_doc=1, k=2) == 0.0
    assert recall_at_k(arr, number_relevant_doc=1, k=4) == 1.0
    assert recall_at_k(arr, number_relevant_doc=4, k=1) == 0

    arr = np.array([1, 1, 1, 1])
    assert recall_at_k(arr, number_relevant_doc=4, k=2) == 0.5
    assert recall_at_k(arr, number_relevant_doc=4, k=4) == 1.0

    arr = np.array([1, 0, 1, 0])
    assert recall_at_k(arr, number_relevant_doc=2, k=2) == 0.5
    assert recall_at_k(arr, number_relevant_doc=2, k=3) == 1.0


def test_average_precision():

    relevance_query_2 = [0, 1, 0, 1, 1, 1, 1]
    result = average_precision(relevance_query_2)
    assert np.isclose(result, 0.5961904, atol=1e-6)

    relevance_all = [1, 1, 1, 1]
    assert average_precision(relevance_all) == 1.0

    relevance_none = [0, 0, 0, 0]
    assert average_precision(relevance_none) == 0.0

    relevance_last = [0, 0, 0, 1]
    assert average_precision(relevance_last) == 0.25


def test_mean_average_precision():
    querys_matrix = np.array([
        [0, 1, 0, 1, 1, 1, 1], 
        [1, 1, 1, 1], 
        [0, 0, 0, 0], 
        [0, 0, 0, 1]
        ], dtype=object
    )

    result = mean_average_precision(querys_matrix)
    expected = np.mean([0.5961904, 1.0, 0.0, 0.25])
    assert np.isclose(result, expected, atol=1e-6)
    querys_matrix = [
        [0, 1, 0, 1, 1, 1, 1],
        [1, 1, 1, 1],
        [0, 0, 0, 0],
        [0, 0, 0, 1, 0, 1],
    ]
    result = mean_average_precision(querys_matrix)
    assert np.isclose(result, np.float64(0.471964285714), atol=1e-6)


test_precision()
print("Se paso la prueba de precision")
test_precision_at_k()
print("Se paso la prueba de precision at k")
test_recall_at_k()
print("Se paso la prueba de recall at k")
test_average_precision()
print("se paso la prueba de average precision")
test_mean_average_precision()
print("Se paso el test de mean average precision")

Se paso la prueba de precision
Se paso la prueba de precision at k
Se paso la prueba de recall at k
se paso la prueba de average precision
Se paso el test de mean average precision
